In [37]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tweepy
import time
import json
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tweepy
import time
import json
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

In [38]:
from config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)

In [39]:
# PlotBot user name (testing with PlotBlot5)
plot_bot = "@CNN"

In [40]:
# Create a function to find the user to analyze
def FindUser():
    
    # Use Tweepy to Authenticate our access
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

    # Get tweets
    public_tweets = api.search(plot_bot, count = 1)['statuses'][0]
    user = "@" + public_tweets['entities']['user_mentions'][1]['screen_name']
    
    if "Analyze" in public_tweets['text']:
        user = "@" + public_tweets['entities']['user_mentions'][1]['screen_name']

    # Get screen name of person who requested analysis
    requesting_user = "@" + public_tweets['user']['screen_name']

    # Return value
    return user, requesting_user

In [41]:
# Create a function to get the tweets of the target user
def GetTweets(target_user):

    # Use Tweepy to Authenticate our access
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

    # Create an empty list for tweets
    user_tweets = []

    # Loop through 25 pages
    for x in range(25):

        # Get tweets
        public_tweets = api.user_timeline(target_user, page = x)

        # Append text of each tweet
        for tweet in public_tweets:
            user_tweets.append(tweet['text'])

    # Return list of tweets
    return user_tweets

In [42]:
# Create a function to analyze list of tweets
def AnalyzeTweets(user_tweets = []):

    # Create a list of compound scores and for tweets ago
    compound_list = []
    tweets_ago = []
    tweet_count = 0

    # Analyze tweets
    for tweet in user_tweets:
        compound = analyzer.polarity_scores(tweet)["compound"]
        compound_list.append(compound)
        tweet_count -= 1
        tweets_ago.append(tweet_count)

    # Return compound scores
    return compound_list, tweets_ago

In [43]:
# Create a function to plot tweet sentiments
def plotting(target_user, tweets_ago = [], compound_senti = []):
    
    # Scatter Plot for sentiment
    plt.plot(tweets_ago, compound_senti, '-o', linewidth=0.5, label=target_user)
    plt.title("Sentiment Analysis of Tweets (%s)" % (time.strftime("%m/%d/%Y")))
    plt.xlabel("Tweets Ago")
    plt.ylabel("Tweet Polarity")
    plt.legend(bbox_to_anchor=(1,0.95), title='Tweets', fontsize='medium')

    plt.savefig("Sentiment_Analysis.png", bbox_inches='tight')

In [44]:
# Create a function to tweet out graph
def TweetOut(target_user, req_user):

    # Use Tweepy to Authenticate our access
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

    # Update status
    api.update_with_media("Sentiment_Analysis.png", "New Tweet Analysis: %s (Thx %s!!)" % (target_user, req_user))

In [45]:
# A list to make sure I don't analyze the same user again
my_mentions = []

while(True):

    # Use Tweepy to Authenticate our access
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
    
    # Get target user
    target_user = FindUser()[0]
    req_user = FindUser()[1]
    
    # Testing whether user has already been analyzed
    if target_user in my_mentions:
        print("%s has already been analyzed!" % target_user)
    else:
        print("Starting analysis of %s" % target_user)

        # Adding user to by analyzed to my_mentions
        my_mentions.append(target_user)

        # Get and analyze tweets
        tweets_analyze = GetTweets(target_user)
        analysis = AnalyzeTweets(tweets_analyze)
        print("Tweets analyzed")

        #Plot the tweets
        plotting(target_user, analysis[1], analysis[0])
        print("Graph plotted")

        # Tweet the image
        TweetOut(target_user, req_user)
        print("Image tweeted")

        print("Analysis Completed!")

    # Checking for new tweets every 5 minutes
    time.sleep(300)

IndexError: list index out of range